# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import sys
import pandasql as ps
import fuzzy_pandas as fpd

# Inspect Oscar Nominees dataset

In [2]:
oscar_df = pd.read_csv("the_oscar_award.csv")
print(oscar_df.shape)

(10759, 7)


In [3]:
oscar_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


#### Reduce timeline (Ceremony 1950-2020)

In [4]:
oscar_df = oscar_df[(oscar_df['year_ceremony'] > 1949) & (oscar_df['year_ceremony'] < 2021)]

In [5]:
oscar_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
2285,1949,1950,22,ACTOR,Broderick Crawford,All the King's Men,True
2286,1949,1950,22,ACTOR,Kirk Douglas,Champion,False
2287,1949,1950,22,ACTOR,Gregory Peck,Twelve O'Clock High,False
2288,1949,1950,22,ACTOR,Richard Todd,The Hasty Heart,False
2289,1949,1950,22,ACTOR,John Wayne,Sands of Iwo Jima,False


#### Filter category for our analysis (Best Picture)

In [6]:
# Check Unique ceremony: df['category'].unique()

oscar_df = oscar_df[oscar_df['category'].isin(['BEST MOTION PICTURE','BEST PICTURE', 'best picture'])]

In [7]:
oscar_df.shape

(398, 7)

In [8]:
oscar_df.dtypes

year_film         int64
year_ceremony     int64
ceremony          int64
category         object
name             object
film             object
winner           object
dtype: object

# Add Rotten Tomatoes Ratings information

#### First Dataset

In [9]:
rt1 = pd.read_csv("rotten_tomatoes_movies.csv")
print(rt1.shape)

(17712, 22)


In [10]:
# rt1.head()

In [11]:
rt1['movieyear'] = rt1['original_release_date'].str[:4].copy()

In [12]:
rt1.head(2)

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,movieyear
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76,2010
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,2012-09-04,90.0,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19,2010


In [1]:
#rt1.dtypes

#### Merge rotten tomatoes(rt1) df to oscar nominees

In [14]:
oscar_df["year_film"] = oscar_df["year_film"].astype(str)
rt1["movieyear"] = rt1["movieyear"].astype(str)

oscar_nominees = fpd.fuzzy_merge(oscar_df, rt1,
                        left_on=['film', 'year_film'],
                        right_on=['movie_title', 'movieyear'],
                        threshold=0.95,
                        method = 'jaro',
                        join='left-outer',
                        ignore_case=True,
                        ignore_titles=True,
                        ignore_nonalpha=True,
                        ignore_nonlatin=True,
                        ignore_order_words=True,
                        ignore_order_letters=True).sort_values(by='year_film', ascending=True).reset_index()

In [15]:
oscar_nominees.shape

(400, 31)

In [16]:
oscar_nominees.head()

,index,year_film,year_ceremony,ceremony,category,name,film,winner,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,movieyear
0,0,1949,1950,22,BEST MOTION PICTURE,Robert Rossen Productions,All the King's Men,True,m/1000654-all_the_kings_men,All the King's Men,Drama about the rise and fall of a corrupt sou...,Broderick Crawford is spellbinding as politici...,PG,"Classics, Drama",Robert Rossen,"Robert Rossen, Robert Penn Warren","Broderick Crawford, Mercedes McCambridge, John...",1949-11-08,2013-05-01,109.0,Sony Pictures Home Entertainment,Certified-Fresh,95.0,42.0,Upright,78.0,4866.0,9,40,2,1949
1,3,1949,1950,22,BEST MOTION PICTURE,20th Century-Fox,Twelve O'Clock High,False,m/1022172-twelve_oclock_high,Twelve O'Clock High,"In 1942, an American Air Force unit stationed ...","Twelve O'Clock High is a high-stakes, high-ten...",NR,"Action & Adventure, Classics, Drama",Henry King,"Beirne Lay, Lt. Beirne Lay Jr., Sy Bartlett","Gregory Peck, Hugh Marlowe, Dean Jagger, Gary ...",1949-01-01,2017-03-13,132.0,Twentieth Century Fox Home Entertainment,Fresh,96.0,25.0,Upright,87.0,6019.0,5,24,1,1949
2,349,1949,1950,22,BEST MOTION PICTURE,Paramount,The Heiress,False,,,,,,,,,,,,,,,,,,,,,,,
3,1,1949,1950,22,BEST MOTION PICTURE,Metro-Goldwyn-Mayer,Battleground,False,m/1001816-battleground,Battleground,Members of the U.S. Army's 101st Airborne Divi...,NaN,NR,"Action & Adventure, Classics, Drama",William Wellman,Robert Pirosh,"Van Johnson, Ricardo Montalban, John Hodiak, G...",1949-11-09,2008-08-18,118.0,MGM,Fresh,75.0,8.0,Upright,82.0,2737.0,1,6,2,1949
4,2,1949,1950,22,BEST MOTION PICTURE,20th Century-Fox,A Letter to Three Wives,False,m/letter_to_three_wives,A Letter to Three Wives,"Deborah Bishop, Lora Mae Hollingsway and Rita ...",NaN,NR,"Classics, Comedy, Drama, Mystery & Suspense, R...",Joseph L. Mankiewicz,"Joseph L. Mankiewicz, Vera Caspary","Jeanne Crain, Ann Sothern, Linda Darnell, Kirk...",1949-01-20,2015-11-25,103.0,Fox,Fresh,94.0,18.0,Upright,86.0,3781.0,2,17,1,1949


#### Deal with any duplicated row

In [17]:
oscar_nominees[oscar_nominees['film'].duplicated(keep=False) == True]

,index,year_film,year_ceremony,ceremony,category,name,film,winner,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,movieyear
18,16,1952,1953,25,BEST MOTION PICTURE,Romulus Films,Moulin Rouge,False,m/1014294-moulin_rouge,Moulin Rouge,"Visually exquisite biopic of Toulouse-Lautrec,...",NaN,NR,"Classics, Drama, Musical & Performing Arts",John Huston,"Anthony Veiller, John Huston","José Ferrer, Colette Marchand, Zsa Zsa Gabor, ...",1952-12-23,2017-04-13,123.0,United Artists,Fresh,78.0,9.0,Upright,90.0,68249.0,1,7,2,1952
263,227,2001,2002,74,BEST PICTURE,"Martin Brown, Baz Luhrmann and Fred Baron, Pro...",Moulin Rouge,False,m/moulin_rouge_2001,Moulin Rouge!,A celebration of love and creative inspiration...,"A love-it-or-hate-it experience, Moulin Rouge ...",PG-13,"Drama, Musical & Performing Arts, Romance",Baz Luhrmann,"Baz Luhrmann, Craig Pearce","Nicole Kidman, Ewan McGregor, John Leguizamo, ...",2001-05-16,2015-11-25,126.0,20th Century Fox,Certified-Fresh,76.0,199.0,Upright,89.0,505231.0,44,151,48,2001
338,299,2013,2014,86,BEST PICTURE,"Charles Roven, Richard Suckle, Megan Ellison a...",American Hustle,False,m/the_human_scale_2013,The Human Scale,"Thinkers, architects and urban planners discus...",NaN,NR,Documentary,Andreas Møl Dalsgaard,Andreas Møl Dalsgaard,"Jan Gehl, Robert Adams, He Dongquan, Robert Do...",2013-10-18,2016-08-10,83.0,Autlook Filmsales GmbH,Fresh,64.0,11.0,Upright,84.0,174.0,6,7,4,2013
344,298,2013,2014,86,BEST PICTURE,"Charles Roven, Richard Suckle, Megan Ellison a...",American Hustle,False,m/american_hustle,American Hustle,Irving Rosenfeld (Christian Bale) dabbles in f...,"Riotously funny and impeccably cast, American ...",R,Drama,David O. Russell,"Eric Warren Singer, David O. Russell","Christian Bale, Bradley Cooper, Amy Adams, Jer...",2013-12-20,2014-09-07,138.0,Sony Pictures,Certified-Fresh,92.0,289.0,Upright,74.0,175937.0,55,267,22,2013
361,319,2015,2016,88,BEST PICTURE,"Arnon Milchan, Steve Golin, Alejandro G. Iñárr...",The Revenant,False,m/the_revenant_2015,The Revenant,While exploring the uncharted wilderness in 18...,As starkly beautiful as it is harshly uncompro...,R,"Action & Adventure, Drama","Alejandro González Iñárritu, Manny Palo","Mark L. Smith, Alejandro González Iñárritu","Leonardo DiCaprio, Tom Hardy, Domhnall Gleeson...",2015-12-25,2016-03-22,156.0,20th Century Fox,Certified-Fresh,78.0,397.0,Upright,84.0,105270.0,53,311,86,2015
362,320,2015,2016,88,BEST PICTURE,"Arnon Milchan, Steve Golin, Alejandro G. Iñárr...",The Revenant,False,m/veteran_2015,Veteran,A detective tracks down a successful young man...,NaN,NR,Drama,Seung-wan Ryoo,NaN,"Jung-min Hwang, Man-sik Jeong, In-young Yoo, J...",2015-09-17,2016-11-19,124.0,CJ Entertainment,Fresh,90.0,10.0,Upright,77.0,594.0,2,9,1,2015


In [18]:
# Dropping duplicated rows

oscar_nominees.drop([338,362], axis=0, inplace=True)

In [19]:
oscar_nominees.shape

(398, 31)

In [20]:
# Filter

q = '''
SELECT
year_film, year_ceremony, film, winner AS oscar_winner, content_rating, runtime, tomatometer_status, tomatometer_rating, audience_rating
FROM
oscar_nominees
'''
oscar_nominees = ps.sqldf(q,locals())

In [21]:
oscar_nominees.head()

,year_film,year_ceremony,film,oscar_winner,content_rating,runtime,tomatometer_status,tomatometer_rating,audience_rating
0,1949,1950,All the King's Men,1,PG,109.0,Certified-Fresh,95.0,78.0
1,1949,1950,Twelve O'Clock High,0,NR,132.0,Fresh,96.0,87.0
2,1949,1950,The Heiress,0,,,,,
3,1949,1950,Battleground,0,NR,118.0,Fresh,75.0,82.0
4,1949,1950,A Letter to Three Wives,0,NR,103.0,Fresh,94.0,86.0


In [22]:
# replace empty spaces with nan
oscar_nominees = oscar_nominees.replace(r'^\s*$', np.nan, regex=True)

In [23]:
oscar_nominees.isna().sum()

year_film              0
year_ceremony          0
film                   0
oscar_winner           0
content_rating        51
runtime               52
tomatometer_status    51
tomatometer_rating    51
audience_rating       51
dtype: int64

In [2]:
#oscar_nominees.dtypes

#### Second RT Dataset

In [25]:
rt2 = pd.read_csv("rt_movie_info.csv")
print(rt2.shape)

(13001, 31)


In [26]:
rt2.head(5)

,movie_link,movie_name,rating,genre,directors,writers,movie_info,poster_image,casts,in_theaters_date,on_streaming_date,runtime_in_minutes,studio_name,box_office,critics_consensus,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_want_to_see_count,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count,audience_fresh_top_critics_count,audience_rotten_rotten_critics_count,tomatometer_avg_rating,audience_top_critics_avg_rating,score_sentiment,audience_avg_rating
0,/m/312291718,Tillie's Punctured Romance,NR,"Classics,Comedy","Charles Bennett,Mack Sennett",Hampton Del Ruth,"This Keystone comedy, Charlie Chaplin's 33rd, ...",https://resizing.flixster.com/2cfo_iZmt1DetxOR...,"Charles Chaplin,Marie Dressler,Mabel Normand,C...","Jan 1, 1914","Aug 24, 1999",73.0,Keystone Film Company,NaN,NaN,fresh,83.0,6,spilled,119,44.0,972.0,1,5,1,1,0,6.38,0.0,NEGATIVE,3.21
1,/m/770681144,Tango Tangles,NR,Comedy,Mack Sennett,Mack Sennett,Tango Tangles is an impromptu Keystone comedy ...,https://resizing.flixster.com/AJKmCcDUQqEOdT3n...,"Glen Cavender,Charles Chaplin,Edgar Kennedy,Fr...","Mar 9, 1914",NaN,12.0,Keystone Film Company,NaN,NaN,NaN,25.0,0,spilled,35,25.0,183.0,0,0,0,0,0,0.00,0.0,NEGATIVE,2.65
2,/m/20297,Cabiria,NR,"Art House & International,Classics,Drama",Giovanni Pastrone,Gabriele D'Annunzio,The ancient conflict between Rome and Carthage...,https://resizing.flixster.com/NA4TdKSRbmLNhJKw...,"Alex Bernard,Italia Almirante-Manzini,Lidia Qu...","Jun 1, 1914","Nov 7, 2000",181.0,Itala Film,NaN,NaN,fresh,89.0,9,upright,176,69.0,1153.0,4,8,1,4,0,7.75,10.0,POSITIVE,3.70
3,/m/770680394,The Magic Cloak of Oz,NR,"Action & Adventure,Drama,Kids & Family,Science...",J. Farrell MacDonald,L. Frank Baum,Author L. Frank Baum adapts his own original s...,https://resizing.flixster.com/qgZlce0CF5nUrmUX...,"Violet MacMillan,Vivian Reed,Mildred Harris,Fr...","Sep 28, 1914","May 22, 2001",45.0,NaN,NaN,NaN,NaN,NaN,0,spilled,0,NaN,NaN,0,0,0,0,0,0.00,0.0,NaN,NaN
4,/m/21368,Regeneration,NR,"Classics,Drama",Raoul Walsh,"Raoul Walsh,Carl Harbaugh",Noteworthy as the first ever feature-length cr...,https://resizing.flixster.com/1aKm46hSGRdsc_Fs...,"Rockliffe Fellowes,Anna Q. Nilsson,William She...","Sep 13, 1915","Nov 27, 2001",61.0,Fox Film Corporation,NaN,NaN,fresh,100.0,5,upright,54,65.0,401.0,2,5,0,2,0,9.17,0.0,POSITIVE,3.50


In [27]:
rt2['movie_year'] = rt2['in_theaters_date'].str[-4:]

In [28]:
# Filter

q = '''
SELECT
movie_name, movie_year, rating AS c_rating, runtime_in_minutes AS run_in_min, tomatometer_status AS tom_status, tomatometer_rating AS tom_rating, audience_rating AS aud_rating
FROM rt2
WHERE movie_year BETWEEN 1949 AND 2021
'''
rt2 = ps.sqldf(q,locals())

In [29]:
rt2.head()

,movie_name,movie_year,c_rating,run_in_min,tom_status,tom_rating,aud_rating
0,It Happens Every Spring,1949,NR,87.0,fresh,83.0,71.0
1,Late Spring,1949,G,107.0,fresh,100.0,93.0
2,The Legend Of The Lone Ranger,1949,G,NaN,None,NaN,NaN
3,She Wore a Yellow Ribbon,1949,G,103.0,fresh,95.0,80.0
4,Shockproof,1949,NR,79.0,fresh,89.0,47.0


In [30]:
# Merge with oscars dataframe

oscar_nominees["year_film"] = oscar_nominees["year_film"].astype(str)
rt2["movie_year"] = rt2["movie_year"].astype(str)

oscar_nominees = fpd.fuzzy_merge(oscar_nominees, rt2,
                        left_on=['film', 'year_film'],
                        right_on=['movie_name', 'movie_year'],
                        threshold=0.99,
                        method = 'jaro',
                        join='left-outer',
                        ignore_case=True,
                        ignore_titles=True,
                        ignore_nonalpha=True,
                        ignore_nonlatin=True,
                        ignore_order_words=True,
                        ignore_order_letters=True).sort_values(by='year_film', ascending=True).reset_index()

In [31]:
oscar_nominees.shape

(543, 17)

In [32]:
# Check for duplicates

oscar_nominees[oscar_nominees['film'].duplicated(keep=False) == True]

,index,year_film,year_ceremony,film,oscar_winner,content_rating,runtime,tomatometer_status,tomatometer_rating,audience_rating,movie_name,movie_year,c_rating,run_in_min,tom_status,tom_rating,aud_rating
18,382,1952,1953,Moulin Rouge,0,NR,123.0,Fresh,78.0,90.0,,,,,,,
22,3,1953,1954,Roman Holiday,0,NR,119.0,Certified-Fresh,97.0,93.0,Roman Holiday,1953,NR,118.0,certified_fresh,98.0,94.0
23,4,1953,1954,Roman Holiday,0,NR,119.0,Certified-Fresh,97.0,93.0,Roman Holiday,1953,NR,118.0,certified_fresh,98.0,94.0
24,5,1953,1954,Shane,0,G,118.0,Fresh,97.0,81.0,Shane,1953,G,118.0,fresh,97.0,81.0
25,6,1953,1954,Shane,0,G,118.0,Fresh,97.0,81.0,Shane,1953,G,118.0,fresh,97.0,81.0
28,7,1954,1955,On the Waterfront,1,NR,108.0,Certified-Fresh,99.0,95.0,On the Waterfront,1954,NR,108.0,certified_fresh,99.0,95.0
29,8,1954,1955,On the Waterfront,1,NR,108.0,Certified-Fresh,99.0,95.0,On the Waterfront,1954,NR,108.0,certified_fresh,99.0,95.0
30,9,1954,1955,Seven Brides for Seven Brothers,0,G,103.0,Fresh,88.0,87.0,Seven Brides for Seven Brothers,1954,G,105.0,fresh,88.0,87.0
31,10,1954,1955,Seven Brides for Seven Brothers,0,G,103.0,Fresh,88.0,87.0,Seven Brides for Seven Brothers,1954,G,105.0,fresh,88.0,87.0
43,14,1956,1957,Giant,0,G,201.0,Certified-Fresh,95.0,87.0,Giant,1956,G,197.0,certified_fresh,95.0,87.0


In [33]:
# Drop duplicates 

oscar_nominees = oscar_nominees.drop_duplicates(subset=["year_film", "film", "movie_name", "movie_year"])
oscar_nominees.shape

(398, 17)

#### Fill NaNs and Drop extra columns

In [35]:
oscar_nominees = oscar_nominees.replace(r'^\s*$', np.nan, regex=True)
oscar_nominees.isna().sum()

index                   0
year_film               0
year_ceremony           0
film                    0
oscar_winner            0
content_rating         51
runtime                52
tomatometer_status     51
tomatometer_rating     51
audience_rating        51
movie_name            176
movie_year            176
c_rating              176
run_in_min            176
tom_status            176
tom_rating            176
aud_rating            176
dtype: int64

In [36]:
# Clean Ratings 

def cleanratings(row):
    if pd.notnull(row['tomatometer_rating']) & pd.notnull(row['tom_rating']):
        return row['tomatometer_rating']
    elif pd.notnull(row['tomatometer_rating']) & pd.isna(row['tom_rating']):
        return row['tomatometer_rating']
    elif pd.isna(row['tomatometer_rating']) & pd.notnull(row['tom_rating']):
        return row['tom_rating']
    else:
        return np.nan
    
def cleanaudience(row):
    if pd.notnull(row['audience_rating']) & pd.notnull(row['aud_rating']):
        return row['audience_rating']
    elif pd.notnull(row['audience_rating']) & pd.isna(row['aud_rating']):
        return row['audience_rating']
    elif pd.isna(row['audience_rating']) & pd.notnull(row['aud_rating']):
        return row['aud_rating']
    else:
        return np.nan
    
def cleanstatus(row):
    if pd.notnull(row['tomatometer_status']) & pd.notnull(row['tom_status']):
        return row['tomatometer_status']
    elif pd.notnull(row['tomatometer_status']) & pd.isna(row['tom_status']):
        return row['tomatometer_status']
    elif pd.isna(row['tomatometer_status']) & pd.notnull(row['tom_status']):
        return row['tom_status']
    else:
        return np.nan

def cleanruntime(row):
    if pd.notnull(row['runtime']) & pd.notnull(row['run_in_min']):
        return row['runtime']
    elif pd.notnull(row['runtime']) & pd.isna(row['run_in_min']):
        return row['runtime']
    elif pd.isna(row['runtime']) & pd.notnull(row['run_in_min']):
        return row['run_in_min']
    else:
        return np.nan
    
def cleancr(row):
    if pd.notnull(row['content_rating']) & pd.notnull(row['c_rating']):
        return row['content_rating']
    elif pd.notnull(row['content_rating']) & pd.isna(row['c_rating']):
        return row['content_rating']
    elif pd.isna(row['content_rating']) & pd.notnull(row['c_rating']):
        return row['c_rating']
    else:
        return np.nan

In [37]:
oscar_nominees['rt_rating'] = oscar_nominees.apply(cleanratings, axis=1)
oscar_nominees['rt_audience_score'] = oscar_nominees.apply(cleanaudience, axis=1)
oscar_nominees['rt_status'] = oscar_nominees.apply(cleanstatus, axis=1)
oscar_nominees['runtime_in_mins'] = oscar_nominees.apply(cleanruntime, axis=1)
oscar_nominees['movie_rating'] = oscar_nominees.apply(cleancr, axis=1)

In [38]:
# Drop duplicate columns

q = '''
SELECT
year_film, year_ceremony, film, oscar_winner, movie_rating, rt_rating, rt_audience_score, rt_status, runtime_in_mins
FROM oscar_nominees
'''
oscar_nominees = ps.sqldf(q,locals())
oscar_nominees.head()

In [40]:
oscar_nominees.isna().sum()

year_film             0
year_ceremony         0
film                  0
oscar_winner          0
movie_rating         35
rt_rating            35
rt_audience_score    35
rt_status            35
runtime_in_mins      36
dtype: int64

# Add IMDB Ratings information

In [41]:
imdb1 = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False)
imdb1.shape

(9699587, 9)

In [42]:
imdb1.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [43]:
imdb1_b = pd.read_csv('title.ratings.tsv', sep='\t', low_memory=False)
imdb1_b.shape

(1290787, 3)

In [44]:
imdb1_b.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1959
1,tt0000002,5.8,263
2,tt0000003,6.5,1798
3,tt0000004,5.6,179
4,tt0000005,6.2,2596


#### Inner Join both IMDB dataframes

In [45]:
imdb1_merge = pd.merge(imdb1, imdb1_b, how='inner', on = 'tconst')
imdb1_merge.shape

(1290787, 11)

In [46]:
imdb1_merge.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1959
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.8,263
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1798
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.6,179
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2596


In [47]:
imdb1_merge.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult            object
startYear          object
endYear            object
runtimeMinutes     object
genres             object
averageRating     float64
numVotes            int64
dtype: object

In [48]:
# Filter

q = '''
SELECT primaryTitle, originalTitle, startYear, averageRating AS imdb_rating, runtimeMinutes FROM imdb1_merge 
WHERE (startYear BETWEEN 1949 AND 2021)
AND (titleType = "movie");
'''
imdb1_merge = ps.sqldf(q,locals())

In [49]:
imdb1_merge.shape

(256651, 5)

In [50]:
imdb1_merge.head()

,primaryTitle,originalTitle,startYear,imdb_rating,runtimeMinutes
0,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,2021,6.7,133
1,La tierra de los toros,La tierra de los toros,2000,5.3,60
2,Dama de noche,Dama de noche,1993,6.1,102
3,Arizona Territory,Arizona Territory,1950,6.0,56
4,Lebbra bianca,Lebbra bianca,1951,5.3,100


In [51]:
# Merge with our oscars dataframe

oscar_nominees["year_film"] = oscar_nominees["year_film"].astype(str)
imdb1_merge["startYear"] = imdb1_merge["startYear"].astype(str)

oscar_nominees = fpd.fuzzy_merge(oscar_nominees, imdb1_merge,
                        left_on=['film', 'year_film'],
                        right_on=['primaryTitle', 'startYear'],
                        threshold=0.99,
                        method = 'jaro',
                        join='left-outer',
                        ignore_case=True,
                        ignore_titles=True,
                        ignore_nonalpha=True,
                        ignore_nonlatin=True,
                        ignore_order_words=True,
                        ignore_order_letters=True).sort_values(by='year_film', ascending=True).reset_index()

In [52]:
oscar_nominees.shape

(402, 15)

In [53]:
# Find duplicates

oscar_nominees[oscar_nominees['film'].duplicated(keep=False) == True]

,index,year_film,year_ceremony,film,oscar_winner,movie_rating,rt_rating,rt_audience_score,rt_status,runtime_in_mins,primaryTitle,originalTitle,startYear,imdb_rating,runtimeMinutes
18,18,1952,1953,Moulin Rouge,0,NR,78.0,90.0,Fresh,123.0,Moulin Rouge,Moulin Rouge,1952,7.0,119
51,53,1959,1960,The Diary of Anne Frank,0,PG,79.0,77.0,Fresh,156.0,The Diary of Anne Frank,Anne no nikki,1995,6.7,102
55,52,1959,1960,The Diary of Anne Frank,0,PG,79.0,77.0,Fresh,156.0,The Diary of Anne Frank,The Diary of Anne Frank,1959,7.4,180
234,223,1995,1996,Apollo 13,0,PG,96.0,87.0,Certified-Fresh,140.0,Apollo 13,Apollo 13,1995,8.4,49
235,222,1995,1996,Apollo 13,0,PG,96.0,87.0,Certified-Fresh,140.0,Apollo 13,Apollo 13,1995,7.7,140
266,249,2001,2002,Moulin Rouge,0,PG-13,76.0,89.0,Certified-Fresh,126.0,Moulin Rouge!,Moulin Rouge!,2001,7.6,127
327,311,2011,2012,The Artist,1,PG-13,95.0,87.0,Certified-Fresh,100.0,The Artist,The Artist,2011,8.2,100
328,310,2011,2012,The Artist,1,PG-13,95.0,87.0,Certified-Fresh,100.0,The Artist,The Artist,2011,7.9,100
363,345,2015,2016,Spotlight,1,R,97.0,93.0,Certified-Fresh,128.0,Spotlight,Spotlight,2015,8.1,129
364,346,2015,2016,Spotlight,1,R,97.0,93.0,Certified-Fresh,128.0,Spotlight,Spotlight,2015,7.8,99


In [54]:
# Drop duplicated rows 

oscar_nominees.drop([51, 234, 327, 364], axis=0, inplace=True)
oscar_nominees.shape

(398, 15)

In [56]:
# Check NaNs

oscar_nominees = oscar_nominees.replace(r'^\s*$', np.nan, regex=True)
oscar_nominees.isna().sum()

index                 0
year_film             0
year_ceremony         0
film                  0
oscar_winner          0
movie_rating         35
rt_rating            35
rt_audience_score    35
rt_status            35
runtime_in_mins      36
primaryTitle         17
originalTitle        17
startYear            17
imdb_rating          17
runtimeMinutes       17
dtype: int64

In [57]:
# Clean runtime column

def cleanruntimemins(row):
    if pd.notnull(row['runtime_in_mins']) & pd.notnull(row['runtimeMinutes']):
        return row['runtimeMinutes']
    elif pd.notnull(row['runtime_in_mins']) & pd.isna(row['runtimeMinutes']):
        return row['runtime_in_mins']
    elif pd.isna(row['runtime_in_mins']) & pd.notnull(row['runtimeMinutes']):
        return row['runtimeMinutes']
    else:
        return np.nan

oscar_nominees['runtime_mins'] = oscar_nominees.apply(cleanruntimemins, axis=1)

In [58]:
# Drop duplicated column

q = '''
SELECT
year_film, year_ceremony, film, oscar_winner, movie_rating, rt_rating, rt_audience_score, rt_status, imdb_rating, runtime_mins 
FROM oscar_nominees
'''
oscar_nominees = ps.sqldf(q,locals())

In [59]:
oscar_nominees.shape

(398, 10)

In [3]:
# oscar_nominees.head()

In [61]:
oscar_nominees.isna().sum()

year_film             0
year_ceremony         0
film                  0
oscar_winner          0
movie_rating         35
rt_rating            35
rt_audience_score    35
rt_status            35
imdb_rating          17
runtime_mins         10
dtype: int64

# Add Golden Globes information

In [62]:
golden_globes = pd.read_csv("goldenglobe.csv")
golden_globes.shape

(7991, 7)

In [63]:
golden_globes.head(2)

,year_film,year_award,ceremony,category,nominee,film,win
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True


In [64]:
golden_globes = golden_globes[(golden_globes['year_award'] > 1949) & (golden_globes['year_award'] < 2021)]

In [65]:
# golden_globes['category'].unique()   

In [66]:
golden_globes = golden_globes[golden_globes['category'].isin(['Picture','Best Motion Picture - Drama','Best Motion Picture - Musical or Comedy','Picture - Comedy', 'Picture - Musical'])]

In [67]:
golden_globes.head()

,year_film,year_award,ceremony,category,nominee,film,win
73,1949,1950,7,Picture,All The King's Men,NaN,True
74,1949,1950,7,Picture,Come To The Stable,NaN,False
112,1950,1951,8,Picture,Sunset Boulevard,NaN,True
113,1950,1951,8,Picture,Born Yesterday,NaN,False
114,1950,1951,8,Picture,Cyrano de Bergerac (1950),NaN,False


In [68]:
golden_globes = golden_globes.drop(['film'], axis=1)

In [69]:
golden_globes.shape

(704, 6)

In [70]:
golden_globes.rename(columns={'year_film': 'film_year', 'category': 'film_cat'}, inplace=True)

In [71]:
golden_globes.dtypes

film_year      int64
year_award     int64
ceremony       int64
film_cat      object
nominee       object
win             bool
dtype: object

In [72]:
# Merge golden globes info with oscar dataframe

oscar_nominees["year_ceremony"] = oscar_nominees["year_ceremony"].astype(str)
golden_globes["year_award"] = golden_globes["year_award"].astype(str)

oscar_nominees = fpd.fuzzy_merge(oscar_nominees, golden_globes,
                        left_on=['film', 'year_ceremony'],
                        right_on=['nominee', 'year_award'],
                        threshold=0.91,
                        method = 'jaro',
                        join='left-outer',
                        ignore_case=True,
                        ignore_titles=True,
                        ignore_nonalpha=True,
                        ignore_nonlatin=True,
                        ignore_order_words=True,
                        ignore_order_letters=True).sort_values(by='year_film', ascending=True).reset_index()

In [73]:
oscar_nominees.shape

(398, 17)

In [74]:
# Clean golden globes column

def clean_ggnominee(x):
    if x == "":
        return '0'
    else:
        return '1'
    
def clean_ggwinners(x):
    if x == True:
        return '1'
    else:
        return '0'    

oscar_nominees['goldenglobe_nom'] = oscar_nominees['nominee'].apply(clean_ggnominee)
oscar_nominees['goldenglobe_win'] = oscar_nominees['win'].apply(clean_ggwinners)

In [75]:
# Drop unneeded columns

oscar_nominees = oscar_nominees.drop(['index', 'film_year', 'year_award', 'ceremony', 'film_cat', 'nominee', 'win'], axis=1).copy()

In [76]:
oscar_nominees.head()

,year_film,year_ceremony,film,oscar_winner,movie_rating,rt_rating,rt_audience_score,rt_status,imdb_rating,runtime_mins,goldenglobe_nom,goldenglobe_win
0,1949,1950,All the King's Men,1,PG,95.0,78.0,Certified-Fresh,7.4,110,1,1
1,1949,1950,Battleground,0,NR,75.0,82.0,Fresh,7.4,118,0,0
2,1949,1950,A Letter to Three Wives,0,NR,94.0,86.0,Fresh,7.7,103,0,0
3,1949,1950,The Heiress,0,None,None,None,None,8.1,115,0,0
4,1949,1950,Twelve O'Clock High,0,NR,96.0,87.0,Fresh,7.7,132,0,0


# Add SAG Awards Information

In [77]:
sag = pd.read_csv("sag.csv")
sag.shape

(5759, 5)

In [78]:
# sag['category'].unique()

In [79]:
sag = sag[sag['category'].isin([' CAST IN A MOTION PICTURE', 'CAST IN A MOTION PICTURE'])].copy()

In [80]:
print(sag.shape)
print(sag.head())

(1135, 5)

In [81]:
# Drop duplicate column and rows

sag.drop(['full_name'], axis=1, inplace=True)
sag.drop_duplicates(subset=["show"], inplace=True)
sag.drop([75,103,4725], axis=0, inplace=True)

In [82]:
# Create year column

sag['sag_year'] = sag['year'].str[:4].copy()

In [83]:
# Drop unneeded columns 

sag = sag.drop(['year', 'category'], axis=1)

In [84]:
sag.head()

,show,won,sag_year
0,BOMBSHELL,False,2020
1,THE IRISHMAN,False,2020
2,JOJO RABBIT,False,2020
3,ONCE UPON A TIME...IN HOLLYWOOD,False,2020
4,PARASITE,True,2020


In [85]:
# Merge SAG info with oscar dataframe

oscar_nominees = fpd.fuzzy_merge(oscar_nominees, sag,
                        left_on=['film', 'year_ceremony'],
                        right_on=['show', 'sag_year'],
                        threshold=0.91,
                        method = 'jaro',
                        join='left-outer',
                        ignore_case=True,
                        ignore_titles=True,
                        ignore_nonalpha=True,
                        ignore_nonlatin=True,
                        ignore_order_words=True,
                        ignore_order_letters=True).sort_values(by='year_film', ascending=True)

In [86]:
oscar_nominees.shape

(398, 15)

In [4]:
#oscar_nominees.head()

In [88]:
# Clean sag columns

def clean_sagnom(x):
    if x == "":
        return '0'
    else:
        return '1'
    
def clean_sagwin(x):
    if x == True:
        return '1'
    else:
        return '0'  
    
oscar_nominees['sag_nom'] = oscar_nominees['show'].apply(clean_sagnom)
oscar_nominees['sag_win'] = oscar_nominees['won'].apply(clean_sagwin)

In [90]:
# Drop unneeded columns 

oscar_nominees = oscar_nominees.drop(['show', 'won', 'sag_year'], axis=1).copy()

In [91]:
oscar_nominees.shape

(398, 14)

In [92]:
oscar_nominees.head()

,year_film,year_ceremony,film,oscar_winner,movie_rating,rt_rating,rt_audience_score,rt_status,imdb_rating,runtime_mins,goldenglobe_nom,goldenglobe_win,sag_nom,sag_win
87,1949,1950,A Letter to Three Wives,0,NR,94.0,86.0,Fresh,7.7,103,0,0,0,0
89,1949,1950,Twelve O'Clock High,0,NR,96.0,87.0,Fresh,7.7,132,0,0,0,0
85,1949,1950,All the King's Men,1,PG,95.0,78.0,Certified-Fresh,7.4,110,1,1,0,0
86,1949,1950,Battleground,0,NR,75.0,82.0,Fresh,7.4,118,0,0,0,0
88,1949,1950,The Heiress,0,None,None,None,None,8.1,115,0,0,0,0


# Add BAFTA awards information

In [93]:
bafta = pd.read_csv("bafta_films.csv")
bafta.shape

(4176, 5)

In [94]:
# Filter

q = '''
SELECT year, category, nominee, winner AS baftawinner
FROM bafta
WHERE (category LIKE '%%Film | Film From Any Source%%'
OR category LIKE '%%Film | Film in%%'
OR category LIKE '%%Film | Best Film in%%')
'''
bafta = ps.sqldf(q,locals())

In [5]:
#bafta.head()

In [96]:
bafta.dtypes

year            int64
category       object
nominee        object
baftawinner     int64
dtype: object

In [97]:
# Merge bafta info with oscar dataframe

bafta["year"] = bafta["year"].astype(str)

oscar_nominees = fpd.fuzzy_merge(oscar_nominees, bafta,
                        left_on=['film', 'year_ceremony'],
                        right_on=['nominee', 'year'],
                        threshold=0.91,
                        method = 'jaro',
                        join='left-outer',
                        ignore_case=True,
                        ignore_titles=True,
                        ignore_nonalpha=True,
                        ignore_nonlatin=True,
                        ignore_order_words=True,
                        ignore_order_letters=True).sort_values(by='year_film', ascending=True).reset_index()

In [102]:
# Clean bafta columns

def clean_baftanom(x):
    if x == "":
        return '0'
    else:
        return '1'
    
def clean_baftawin(x):
    if x == 1:
        return '1'
    else:
        return '0'
    
oscar_nominees['bafta_nom'] = oscar_nominees['nominee'].apply(clean_baftanom)
oscar_nominees['bafta_win'] = oscar_nominees['baftawinner'].apply(clean_baftawin)

In [104]:
# Drop unneeded columns

oscar_nominees = oscar_nominees.drop(['index', 'category', 'nominee', 'baftawinner', 'year'], axis=1).copy()

In [106]:
oscar_nominees.isna().sum()

year_film             0
year_ceremony         0
film                  0
oscar_winner          0
movie_rating         35
rt_rating            35
rt_audience_score    35
rt_status            35
imdb_rating          17
runtime_mins         10
goldenglobe_nom       0
goldenglobe_win       0
sag_nom               0
sag_win               0
bafta_nom             0
bafta_win             0
dtype: int64

# View final movie dataset for additional features and fill NaN's

In [107]:
data = pd.read_csv("oscardata_bestpicture.csv")
data.shape
#data.head()

(333, 60)

#### Select Useful Fields

In [110]:
q = '''
SELECT Nominee AS oscar_nom, Year AS award_year, Rating_IMDB, Rating_rtaudience, Rating_rtcritic, MPAA_rating, Nom_SAG_bestcast, Win_SAG_bestcast, Nom_BAFTA, Win_BAFTA, Nom_GoldenGlobe_bestcomedy, Win_GoldenGlobe_bestcomedy, Nom_GoldenGlobe_bestdrama, Win_GoldenGlobe_bestdrama
FROM data
'''
data = ps.sqldf(q,locals())

In [244]:
data.head()

,oscar_nom,award_year,Rating_IMDB,Rating_rtaudience,Rating_rtcritic,MPAA_rating,Nom_SAG_bestcast,Win_SAG_bestcast,Nom_BAFTA,Win_BAFTA,Nom_GoldenGlobe_bestcomedy,Win_GoldenGlobe_bestcomedy,Nom_GoldenGlobe_bestdrama,Win_GoldenGlobe_bestdrama
0,The Apartment,1961,8.3,94,93,APPROVED,0,0,1,1,1,1,0,0
1,The Alamo,1961,6.9,64,50,UNRATED,0,0,0,0,0,0,0,0
2,Elmer Gantry,1961,7.9,86,96,APPROVED,0,0,1,0,0,0,1,0
3,Sons and Lovers,1961,7.3,54,75,None,0,0,0,0,0,0,1,0
4,The Sundowners,1961,7.2,62,78,UNRATED,0,0,1,0,0,0,0,0


In [113]:
data.dtypes

oscar_nom                      object
award_year                      int64
Rating_IMDB                   float64
Rating_rtaudience               int64
Rating_rtcritic                 int64
MPAA_rating                    object
Nom_SAG_bestcast                int64
Win_SAG_bestcast                int64
Nom_BAFTA                       int64
Win_BAFTA                       int64
Nom_GoldenGlobe_bestcomedy      int64
Win_GoldenGlobe_bestcomedy      int64
Nom_GoldenGlobe_bestdrama       int64
Win_GoldenGlobe_bestdrama       int64
dtype: object

In [129]:
# Merge movie data with oscar dataframe

data["award_year"] = data["award_year"].astype(str)
data_merge = fpd.fuzzy_merge(oscar_nominees, data,
                        left_on=['film', 'year_ceremony'],
                        right_on=['oscar_nom', 'award_year'],
                        threshold=0.9,
                        method = 'jaro',
                        join='left-outer',
                        ignore_case=True,
                        ignore_titles=True,
                        ignore_nonalpha=True,
                        ignore_nonlatin=True,
                        ignore_order_words=True,
                        ignore_order_letters=True).sort_values(by='year_film', ascending=True).reset_index()

In [131]:
data_merge.shape

(398, 31)

In [132]:
data_merge.head(1)

,index,year_film,year_ceremony,film,oscar_winner,movie_rating,rt_rating,rt_audience_score,rt_status,imdb_rating,runtime_mins,goldenglobe_nom,goldenglobe_win,sag_nom,sag_win,bafta_nom,bafta_win,oscar_nom,award_year,Rating_IMDB,Rating_rtaudience,Rating_rtcritic,MPAA_rating,Nom_SAG_bestcast,Win_SAG_bestcast,Nom_BAFTA,Win_BAFTA,Nom_GoldenGlobe_bestcomedy,Win_GoldenGlobe_bestcomedy,Nom_GoldenGlobe_bestdrama,Win_GoldenGlobe_bestdrama
0,331,1949,1950,A Letter to Three Wives,0,NR,94.0,86.0,Fresh,7.7,103,0,0,0,0,0,0,,,,,,,,,,,,,,


#### Clean NaN's using movies dataset

In [ ]:
#BAFTA
def cleanbaftanom2(row):
    if (row['bafta_nom'] == "0") & (row['Nom_BAFTA'] == "1"):
        return "1"
    elif (row['bafta_nom'] == "1") & (row['Nom_BAFTA'] == "1"):
        return "1"
    elif (row['bafta_nom'] == "1") & (row['Nom_BAFTA'] == "0" or row['Nom_BAFTA'] == ''):
        return "1"
    elif (row['bafta_nom'] == "0") & (row['Nom_BAFTA'] == "0" or row['Nom_BAFTA'] == ''):
        return "0"
    
def cleanbaftawin2(row):
    if (row['bafta_win'] == "0") & (row['Win_BAFTA'] == "1"):
        return "1"
    elif (row['bafta_win'] == "1") & (row['Win_BAFTA'] == "1"):
        return "1"
    elif (row['bafta_win'] == "1") & (row['Win_BAFTA'] == "0" or row['Win_BAFTA'] == ''):
        return "1"
    elif (row['bafta_win'] == "0") & (row['Win_BAFTA'] == "0" or row['Win_BAFTA'] == ''):
        return "0"

#SAG
def cleansagnom2(row):
    if (row['sag_nom'] == "0") & (row['Nom_SAG_bestcast'] == "1"):
        return "1"
    elif (row['sag_nom'] == "1") & (row['Nom_SAG_bestcast'] == "1"):
        return "1"
    elif (row['sag_nom'] == "1") & (row['Nom_SAG_bestcast'] == "0" or row['Nom_SAG_bestcast'] == ''):
        return "1"
    elif (row['sag_nom'] == "0") & (row['Nom_SAG_bestcast'] == "0" or row['Nom_SAG_bestcast'] == ''):
        return "0"
    
def cleansagwin2(row):
    if (row['sag_win'] == "0") & (row['Win_SAG_bestcast'] == "1"):
        return "1"
    elif (row['sag_win'] == "1") & (row['Win_SAG_bestcast'] == "1"):
        return "1"
    elif (row['sag_win'] == "1") & (row['Win_SAG_bestcast'] == "0" or row['Win_SAG_bestcast'] == ''):
        return "1"
    elif (row['sag_win'] == "0") & (row['Win_SAG_bestcast'] == "0" or row['Win_SAG_bestcast'] == ''):
        return "0"  
    
#Golden Globes
def cleanggnom2(row):
    if (row['goldenglobe_nom'] == "0") & (row['Nom_GoldenGlobe_bestcomedy'] == "1"):
        return "1"
    elif (row['goldenglobe_nom'] == "0") & (row['Nom_GoldenGlobe_bestdrama'] == "1"):
        return "1"
    elif (row['goldenglobe_nom'] == "1") & (row['Nom_GoldenGlobe_bestcomedy'] == "1"):
        return "1"
    elif (row['goldenglobe_nom'] == "1") & (row['Nom_GoldenGlobe_bestdrama'] == "1"):
        return "1"
    elif (row['goldenglobe_nom'] == "1") & (row['Nom_GoldenGlobe_bestcomedy'] == "0" or row['Nom_GoldenGlobe_bestcomedy'] == ''):
        return "1"
    elif (row['goldenglobe_nom'] == "1") & (row['Nom_GoldenGlobe_bestdrama'] == "0" or row['Nom_GoldenGlobe_bestdrama'] == ''):
        return "1"
    elif (row['goldenglobe_nom'] == "0") & (row['Nom_GoldenGlobe_bestcomedy'] == "0" or row['Nom_GoldenGlobe_bestcomedy'] == ''):
        return "0"  
    elif (row['goldenglobe_nom'] == "0") & (row['Nom_GoldenGlobe_bestdrama'] == "0" or row['Nom_GoldenGlobe_bestdrama'] == ''):
        return "0"  

def cleanggwin2(row):
    if (row['goldenglobe_win'] == "0") & (row['Win_GoldenGlobe_bestcomedy'] == "1"):
        return "1"
    elif (row['goldenglobe_win'] == "0") & (row['Win_GoldenGlobe_bestdrama'] == "1"):
        return "1"
    elif (row['goldenglobe_win'] == "1") & (row['Win_GoldenGlobe_bestcomedy'] == "1"):
        return "1"
    elif (row['goldenglobe_win'] == "1") & (row['Win_GoldenGlobe_bestdrama'] == "1"):
        return "1"
    elif (row['goldenglobe_win'] == "1") & (row['Win_GoldenGlobe_bestcomedy'] == "0" or row['Win_GoldenGlobe_bestcomedy'] == ''):
        return "1"
    elif (row['goldenglobe_win'] == "1") & (row['Win_GoldenGlobe_bestdrama'] == "0" or row['Win_GoldenGlobe_bestdrama'] == ''):
        return "1"
    elif (row['goldenglobe_win'] == "0") & (row['Win_GoldenGlobe_bestcomedy'] == "0" or row['Win_GoldenGlobe_bestcomedy'] == ''):
        return "0"  
    elif (row['goldenglobe_win'] == "0") & (row['Win_GoldenGlobe_bestdrama'] == "0" or row['Win_GoldenGlobe_bestdrama'] == ''):
        return "0"  

#Rotten Tomatoes
def cleanrtratings2(row):
    if pd.notnull(row['rt_rating']) & pd.notnull(row['Rating_rtcritic']):
        return row['rt_rating']
    elif pd.notnull(row['rt_rating']) & pd.isna(row['Rating_rtcritic']):
        return row['rt_rating']
    elif pd.isna(row['rt_rating']) & pd.notnull(row['Rating_rtcritic']):
        return row['Rating_rtcritic']
    else:
        return np.nan
    
#Audience Ratings
def cleanaudience2(row):
    if pd.notnull(row['rt_audience_score']) & pd.notnull(row['Rating_rtaudience']):
        return row['rt_audience_score']
    elif pd.notnull(row['rt_audience_score']) & pd.isna(row['Rating_rtaudience']):
        return row['rt_audience_score']
    elif pd.isna(row['rt_audience_score']) & pd.notnull(row['Rating_rtaudience']):
        return row['Rating_rtaudience']
    else:
        return np.nan

#IMDB Ratings
def cleanimdb2(row):
    if pd.notnull(row['imdb_rating']) & pd.notnull(row['Rating_IMDB']):
        return row['imdb_rating']
    elif pd.notnull(row['imdb_rating']) & pd.isna(row['Rating_IMDB']):
        return row['imdb_rating']
    elif pd.isna(row['imdb_rating']) & pd.notnull(row['Rating_IMDB']):
        return row['Rating_IMDB']
    else:
        return np.nan
    
# data_merge['imdb_rating2'] = data_merge.apply(cleanimdb2, axis=1)
# data_merge = data_merge.replace(r'^\s*$', np.nan, regex=True)


In [227]:
# Drop duplicate columns

data_merge = data_merge.drop(['index'], axis=1).copy() 
data_merge = data_merge.drop(['Nom_SAG_bestcast','Win_SAG_bestcast','Nom_BAFTA','Win_BAFTA','Nom_GoldenGlobe_bestcomedy','Win_GoldenGlobe_bestcomedy','Nom_GoldenGlobe_bestdrama','Win_GoldenGlobe_bestdrama','goldenglobe_nom','goldenglobe_win','sag_nom','sag_win','bafta_nom','bafta_win'], axis=1).copy() 
data_merge = data_merge.drop(['oscar_nom','award_year','Rating_IMDB','Rating_rtcritic','Rating_rtaudience','MPAA_rating','rt_rating','rt_audience_score','imdb_rating'], axis=1).copy() 

In [245]:
data_merge.isna().sum()

index                  0
year_film              0
year_ceremony          0
film                   0
oscar_winner           0
movie_rating          35
rt_status             35
runtime_mins          10
bafta_nom2             0
bafta_win2             0
sag_nom2               0
sag_win2               0
gg_nom2                0
gg_win2                0
rt_rating2             0
rt_audience_score2     0
imdb_rating2           0
dtype: int64

# Fill remaining NaN's manually

In [6]:
data_merge[data_merge.isna().any(axis=1)]

In [260]:
# data_merge.loc[8,['movie_rating','rt_status','rt_rating2','rt_audience_score2', 'gg_nom2', 'gg_win2']] = ['Passed', 'Certified-Fresh', 98.0, 95.0, '1', '1']
# data_merge.loc[43,['movie_rating','rt_status','rt_rating2','rt_audience_score2', 'bafta_nom2']] = ['Approved', 'Certified-Fresh', 100.0, 97.0, '1']
# data_merge.loc[57,['movie_rating','rt_status','rt_rating2','rt_audience_score2']] = ['Approved', 'Fresh', 67.0, 50.0]
# data_merge.loc[60,['movie_rating','rt_status','rt_rating2','rt_audience_score2']] = ['Approved', 'Fresh', 80.0, 67.0]
# data_merge.loc[81,['movie_rating','rt_status','rt_rating2','rt_audience_score2']] = ['Approved', 'Fresh', 72.0, 88.0]
# data_merge.loc[118,['movie_rating','rt_status','rt_rating2','rt_audience_score2', 'gg_nom2','runtime_mins']] = ['PG', 'Fresh', 93.0, 87.0, '1', 191]
# data_merge.loc[124,['movie_rating','rt_status','rt_rating2','rt_audience_score2','imdb_rating2','gg_nom2','runtime_mins']] = ['R', 'Fresh', 94.0, 90.0, 8.0, '1', 91]
# data_merge.loc[141,['movie_rating','rt_status','rt_rating2','rt_audience_score2']] = ['PG', 'Fresh', 63.0, 67.0]
# data_merge.loc[142,['movie_rating','rt_status','runtime_mins']] = ['PG', 'Certified-Fresh', 121]
# data_merge.loc[143,['movie_rating','rt_status','rt_rating2','rt_audience_score2']] = ['PG', 'Fresh', 74.0, 70.0]
# data_merge.loc[146,['movie_rating','rt_status']] = ['R', 'Fresh']
# data_merge.loc[156,['movie_rating','rt_status','runtime_mins','gg_nom2']] = ['PG', 'Certified-Fresh',186,'1']
# data_merge.loc[164,['movie_rating','rt_status','runtime_mins','gg_nom2']] = ['R', 'Fresh',104,'1']
# data_merge.loc[169,['movie_rating','rt_status']] = ['PG', 'Fresh']
# data_merge.loc[188,['movie_rating','rt_status','runtime_mins']] = ['NR', 'Fresh',117]
# data_merge.loc[230,['movie_rating','rt_status','runtime_mins','bafta_nom2','bafta_win2']] = ['PG', 'Fresh',108,'1','1']
# data_merge.loc[234,['movie_rating','rt_status']] = ['PG', 'Certified-Fresh']
# data_merge.loc[237,['movie_rating','rt_status']] = ['R', 'Certified-Fresh']
# data_merge.loc[246,['movie_rating','rt_status','runtime_mins','bafta_nom2']] = ['PG-13', 'Certified-Fresh',116,'1']
# data_merge.loc[257,['movie_rating','rt_status']] = ['PG-13','Certified-Fresh']
# data_merge.loc[230,['bafta_win2']] = ['0']
# data_merge.loc[281,['movie_rating','rt_status','runtime_mins']] = ['R','Certified-Fresh',112]
# data_merge.loc[325,['movie_rating','rt_status']] = ['PG-13','Rotten']
# data_merge.loc[331,['movie_rating','rt_status']] = ['R','Certified-Fresh']
# data_merge.loc[348,['movie_rating','rt_status','rt_audience_score2']] = ['R','Certified-Fresh',78.0]
# data_merge.loc[354,['movie_rating','rt_status']] = ['R','Certified-Fresh']
# data_merge.loc[364,['movie_rating','rt_status']] = ['PG','Certified-Fresh']
# data_merge.loc[372,['movie_rating','rt_status']] = ['R','Certified-Fresh']
# data_merge.loc[375,['movie_rating','rt_status']] = ['R','Certified-Fresh']
# data_merge.loc[378,['movie_rating','rt_status']] = ['PG-13','Certified-Fresh']
# data_merge.loc[389,['movie_rating','rt_status','rt_rating2','rt_audience_score2']] = ['R','Certified-Fresh',89.0,88.0]
# data_merge.loc[390,['movie_rating','rt_status','rt_rating2','rt_audience_score2']] = ['PG-13','Certified-Fresh',92.0,98.0]
# data_merge.loc[391,['movie_rating','rt_status','rt_rating2','rt_audience_score2']] = ['PG','Certified-Fresh',95.0,92.0]
# data_merge.loc[395,['movie_rating','rt_status','rt_rating2','rt_audience_score2','gg_nom2']] = ['R','Certified-Fresh',99.0,90.0,'1']
# data_merge.loc[306,['runtime_mins','imdb_rating2','bafta_nom2','sag_nom2']] = [110,7.3,'1','1']
# data_merge.loc[396,['runtime_mins','imdb_rating2']] = [161,7.6]

In [269]:
data_merge.isna().sum()

year_film             0
year_ceremony         0
film                  0
oscar_winner          0
movie_rating          0
rt_status             0
runtime_mins          0
bafta_nom2            0
bafta_win2            0
sag_nom2              0
sag_win2              0
gg_nom2               0
gg_win2               0
rt_rating2            0
rt_audience_score2    0
imdb_rating2          0
dtype: int64

In [325]:
data_merge.shape

(398, 16)

In [324]:
# Filter data as needed

q = '''
SELECT year_film, film, year_ceremony AS award_year, imdb_rating2 AS imdb_rating, rt_rating2 AS rt_rating, 
rt_audience_score2 AS rt_audience_score, rt_status, runtime_mins, movie_rating AS content_rating,
bafta_nom2 AS bafta_nom, bafta_win2 AS bafta_win, sag_nom2 AS sag_nom, sag_win2 AS sag_win, 
gg_nom2 AS gg_nom, gg_win2 AS gg_win, oscar_winner
FROM data_merge
'''
oscar_nominees = ps.sqldf(q,locals())

# Clean Columns

#### Clean Rotten Tomatoes Status

In [328]:
oscar_nominees['rt_status'].value_counts()

Certified-Fresh    268
Fresh              105
Rotten              10
fresh                9
certified_fresh      6
Name: rt_status, dtype: int64

In [329]:
def clean_stat(x):
    x = x.lower()
    if 'certified_fresh' in x:
        return "certified-fresh"
    else:
        return x

oscar_nominees['rt_status'] = list(map(clean_stat, oscar_nominees['rt_status']))

#### Clean Content Rating Column

In [331]:
oscar_nominees['content_rating'].value_counts()

R           156
PG           85
PG-13        76
NR           39
G            37
Approved      4
Passed        1
Name: content_rating, dtype: int64

In [332]:
def clean_rating(x):
    if "Approved" in x:
        return "NR"
    elif "Passed" in x:
        return "NR"
    else:
        return x

oscar_nominees['content_rating'] = list(map(clean_rating, oscar_nominees['content_rating']))

In [342]:
oscar_nominees.head()

,year_film,film,award_year,imdb_rating,rt_rating,rt_audience_score,rt_status,runtime_mins,content_rating,bafta_nom,bafta_win,sag_nom,sag_win,gg_nom,gg_win,oscar_winner
0,1949,A Letter to Three Wives,1950,7.7,94.0,86.0,fresh,103,NR,0,0,0,0,0,0,0
1,1949,Battleground,1950,7.4,75.0,82.0,fresh,118,NR,0,0,0,0,0,0,0
2,1949,The Heiress,1950,8.1,100.0,93.0,fresh,115,NR,0,0,0,0,0,0,0
3,1949,Twelve O'Clock High,1950,7.7,96.0,87.0,fresh,132,NR,0,0,0,0,0,0,0
4,1949,All the King's Men,1950,7.4,95.0,78.0,certified-fresh,110,PG,0,0,0,0,1,1,1


# Add more useful features (DGA, genre and total noms)

In [337]:
data2 = pd.read_csv("oscardata_bestpicture.csv")
data2.shape
data2.head()

,Category,Film,Nominee,Winner,Year,Rating_IMDB,Release_date,Rating_rtaudience,Rating_rtcritic,Oscarstat_totalnoms,Release_Q1,Release_Q2,Release_Q3,Release_Q4,Nom_Oscar_bestdirector,Nom_DGA,Nom_BAFTA,Win_DGA,Win_BAFTA,Nom_GoldenGlobe_bestcomedy,Nom_GoldenGlobe_bestdrama,Win_GoldenGlobe_bestcomedy,Win_GoldenGlobe_bestdrama,Genre_action,Genre_biography,Genre_crime,Genre_comedy,Genre_drama,Genre_horror,Genre_fantasy,Genre_sci-fi,Genre_mystery,Genre_music,Genre_romance,Genre_history,Genre_war,Genre_filmnoir,Genre_thriller,Genre_adventure,Genre_family,Genre_sport,Genre_western,MPAA_rating,MPAA_G,MPAA_PG,MPAA_PG-13,MPAA_R,MPAA_NC-17,Nowin_Criticschoice,Win_Criticschoice,Nonom_Criticschoice,Nom_Criticschoice,Nowin_SAG_bestcast,Win_SAG_bestcast,Nonom_SAG_bestcast,Nom_SAG_bestcast,Nowin_PGA,Win_PGA,Nonom_PGA,Nom_PGA
0,Picture,The Apartment,The Apartment,1,1961,8.3,1960-09-16,94,93,10,0,0,1,0,1,1,1,1,1,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,APPROVED,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Picture,The Alamo,The Alamo,0,1961,6.9,1960-10-24,64,50,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,UNRATED,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Picture,Elmer Gantry,Elmer Gantry,0,1961,7.9,1960-07,86,96,5,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,APPROVED,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Picture,Sons and Lovers,Sons and Lovers,0,1961,7.3,1960-07-22,54,75,7,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Picture,The Sundowners,The Sundowners,0,1961,7.2,1961-02-28,62,78,5,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,UNRATED,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [339]:

data2 = data2[['Nominee','Year','Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_drama','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']].copy()


In [340]:
data2.head()

,Nominee,Year,Oscarstat_totalnoms,Nom_DGA,Win_DGA,Genre_action,Genre_biography,Genre_crime,Genre_comedy,Genre_drama,Genre_horror,Genre_fantasy,Genre_sci-fi,Genre_mystery,Genre_music,Genre_romance,Genre_history,Genre_war,Genre_filmnoir,Genre_thriller,Genre_adventure,Genre_family,Genre_sport,Genre_western
0,The Apartment,1961,10,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,The Alamo,1961,6,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,Elmer Gantry,1961,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Sons and Lovers,1961,7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,The Sundowners,1961,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [341]:
data2.dtypes

Nominee                object
Year                    int64
Oscarstat_totalnoms     int64
Nom_DGA                 int64
Win_DGA                 int64
Genre_action            int64
Genre_biography         int64
Genre_crime             int64
Genre_comedy            int64
Genre_drama             int64
Genre_horror            int64
Genre_fantasy           int64
Genre_sci-fi            int64
Genre_mystery           int64
Genre_music             int64
Genre_romance           int64
Genre_history           int64
Genre_war               int64
Genre_filmnoir          int64
Genre_thriller          int64
Genre_adventure         int64
Genre_family            int64
Genre_sport             int64
Genre_western           int64
dtype: object

In [376]:
# Merge new columns with oscar dataframe

data2["Year"] = data2["Year"].astype(str)
data2_merge = fpd.fuzzy_merge(oscar_nominees, data2,
                        left_on=['film', 'award_year'],
                        right_on=['Nominee', 'Year'],
                        threshold=0.9,
                        method = 'jaro',
                        join='left-outer',
                        ignore_case=True,
                        ignore_titles=True,
                        ignore_nonalpha=True,
                        ignore_nonlatin=True,
                        ignore_order_words=True,
                        ignore_order_letters=True).sort_values(by='year_film', ascending=True).reset_index()

#### Filter year we will be using in analysis (1960-2020)

In [379]:
q = '''
SELECT * FROM
data2_merge
WHERE award_year > 1959
'''
data2_merge = ps.sqldf(q,locals())

In [388]:
data2_merge.shape

(348, 41)

In [7]:
#data2_merge

#### Manually fill in empty data

In [401]:
# data2_merge.loc[0,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [8, 1, 0, 1, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[1,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_mystery','Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [7, 1, 0, 1, 1, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[2,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_adventure','Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_family','Genre_sport','Genre_western']] = [12, 1, 1, 1, 1, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[3,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_biography', 'Genre_family','Genre_action','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_sport','Genre_western']] = [8, 1, 0, 1, 1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[4,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_romance', 'Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [6, 0, 0, 1, 1, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[15,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_comedy','Genre_family','Genre_music','Genre_drama', 'Genre_romance', 'Genre_action', 'Genre_biography','Genre_crime','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_sport','Genre_western']] = [6, 1, 0, 1, 1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[256,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [6, 1, 0, 1, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[282,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_music','Genre_drama', 'Genre_romance','Genre_comedy','Genre_family', 'Genre_action', 'Genre_biography','Genre_crime','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_sport','Genre_western']] = [8, 1, 0, 1, 1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[339,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_comedy', 'Genre_action','Genre_biography','Genre_crime','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [10, 1, 0, 1, 1, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[340,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_action', 'Genre_war','Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [10, 1, 1, 1, 1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[341,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [4, 0, 0, 1, 1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[342,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_romance', 'Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [6, 0, 0, 1, 1, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[343,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_biography','Genre_crime','Genre_action','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [10, 1, 0, 1, 1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[344,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_comedy', 'Genre_war','Genre_action','Genre_biography','Genre_crime','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [6, 1, 0, 1, 1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[345,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_crime','Genre_thriller','Genre_biography','Genre_action','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [11, 0, 0, 1, 1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[346,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama','Genre_thriller', 'Genre_crime','Genre_biography','Genre_action','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_romance','Genre_history','Genre_war','Genre_filmnoir','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [6, 1, 0, 1, 1, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# data2_merge.loc[347,['Oscarstat_totalnoms', 'Nom_DGA', 'Win_DGA', 'Genre_drama', 'Genre_romance', 'Genre_action', 'Genre_biography','Genre_crime','Genre_comedy','Genre_horror','Genre_fantasy','Genre_sci-fi','Genre_mystery','Genre_music','Genre_history','Genre_war','Genre_filmnoir','Genre_thriller','Genre_adventure','Genre_family','Genre_sport','Genre_western']] = [6, 0, 0, 1, 1, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


In [420]:
# data2_merge.loc[39,['Oscarstat_totalnoms','Genre_comedy','Genre_war']] = [4,1,1]
# data2_merge.loc[83,['Oscarstat_totalnoms']] = [9]
# data2_merge.loc[118,['Oscarstat_totalnoms']] = [9]
# data2_merge.loc[157,['Oscarstat_totalnoms','Genre_drama','Genre_crime']] = [7,1,1]
# data2_merge.loc[158,['Oscarstat_totalnoms']] = [6,]
# data2_merge.loc[159,['Oscarstat_totalnoms']] = [12]
# data2_merge.loc[183,['Oscarstat_totalnoms','Genre_drama','Genre_comedy','Genre_biography']] = [5,1,1,1]
# data2_merge.loc[189,['Oscarstat_totalnoms','Genre_drama','Genre_comedy']] = [5,1,1]
# data2_merge.loc[207,['Oscarstat_totalnoms','Genre_drama','Genre_action','Genre_adventure']] = [10,1,1,1]
# data2_merge.loc[214,['Oscarstat_totalnoms']] = [8]
# data2_merge.loc[230,['Oscarstat_totalnoms','Genre_biography','Genre_drama','Genre_history']] = [6,1,1,1]
# data2_merge.loc[270,['Oscarstat_totalnoms','Genre_drama','Genre_adventure','Genre_mystery']] = [2,1,1,1]

#### Drop unneeded columns

In [422]:
oscar_noms = data2_merge.drop(['index', 'Nominee', 'Year'], axis=1).copy()

#### Clean SAG columns

In [424]:
def cleansagnom(row):
    if (row['sag_nom'] == "0") & (row['award_year'] < "1996"):
        return "DE"
    elif (row['sag_nom'] == "0")& (row['award_year'] > "1995"):
        return "0"
    elif (row['sag_nom'] == "1"):
        return "1"
    
def cleansagwin(row):
    if (row['sag_win'] == "0") & (row['award_year'] < "1996"):
        return "DE"
    elif (row['sag_win'] == "0")& (row['award_year'] > "1995"):
        return "0"
    elif (row['sag_win'] == "1"):
        return "1"
    
oscar_noms['sag_nom'] = oscar_noms.apply(cleansagnom, axis=1)
oscar_noms['sag_win'] = oscar_noms.apply(cleansagwin, axis=1)

In [426]:
oscar_noms["sag_win"].value_counts()

DE    180
0     144
1      24
Name: sag_win, dtype: int64

In [10]:
#oscar_noms.head()

# Saved finished dataframe as CSV

In [429]:
oscar_noms.to_csv('oscar.csv', index=False)